<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align = center><font size = 5> Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</font></h1>

#### *Import Libraries*

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
import numpy as np 
import pandas as pd
from pandas.io.json import json_normalize 
import json 
import requests 

from sklearn.cluster import KMeans 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import matplotlib.cm as cm
import matplotlib.colors as colors

import folium 

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge


#### *Scrape link and create dataframe for neighborhoods in Toronto, CA*

In [23]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

text_result = requests.get(url).text
html_parse = BeautifulSoup(text_result, 'html.parser')

neighborhood_info_table = html_parse.find('table', class_ = 'wikitable sortable')
neighborhood_rows = neighborhood_info_table.find_all('tr')

neighborhood_info = []
for row in neighborhood_rows:
    info = row.text.split('\n')[1:-1] 
    neighborhood_info.append(info)
    
neighborhood_info[0:10]

neighborhood_info[0][-1] = 'Neighborhood' 
neighborhood_df = pd.DataFrame(neighborhood_info[1:], columns=neighborhood_info[0])




notassigned_neighborhoods = neighborhood_df.index[neighborhood_df['Borough'] == 'Not assigned']
notassigned_boroughs = neighborhood_df.index[neighborhood_df['Neighborhood'] == 'Not assigned']
notassignedNeighBoroughs = notassigned_neighborhoods & notassigned_boroughs
neighborhood_df.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


#### *Adjust for neighborhoods that are not assigned and create new Dataframe*

In [24]:
notassigned_neighborhoods = neighborhood_df.index[neighborhood_df['Neighborhood'] == 'Not assigned'] 

for index in notassigned_neighborhoods:
    neighborhood_df['Neighborhood'][index] = neighborhood_df['Borough'][index]

neighborhood_df.drop(neighborhood_df.index[notassigned_boroughs], inplace=True)
neighborhood_df.reset_index(drop=True, inplace=True)
neighborhood_df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,Rouge
7,M1B,Scarborough,Malvern
8,M3B,North York,Don Mills North
9,M4B,East York,Woodbine Gardens


#### *Dataframe with Latitude and Longitude Coordinates*

In [19]:
coordinates_url = "http://cocl.us/Geospatial_data"

coordinates = pd.read_csv(coordinates_url)
coordinates.head()

neighborhoods_with_coordinates = neighborhood_df.join(coordinates.set_index('Postal Code'), on='Postcode')

neighborhoods_with_coordinates


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
6,M1B,Scarborough,Rouge,43.806686,-79.194353
7,M1B,Scarborough,Malvern,43.806686,-79.194353
8,M3B,North York,Don Mills North,43.745906,-79.352188
9,M4B,East York,Woodbine Gardens,43.706397,-79.309937


#### *Dataframe grouped by the Borough: North York*

In [5]:
groupbyDF = neighborhoods_with_coordinates[neighborhoods_with_coordinates['Borough'] == 'North York']
groupbyDF

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
8,M3B,North York,Don Mills North,43.745906,-79.352188
13,M6B,North York,Glencairn,43.709577,-79.445073
22,M3C,North York,Flemingdon Park,43.725900,-79.340923
23,M3C,North York,Don Mills South,43.725900,-79.340923
42,M2H,North York,Hillcrest Village,43.803762,-79.363452
43,M3H,North York,Bathurst Manor,43.754328,-79.442259


#### *K-Clustering*

In [14]:
kclusters = 5
canada_clustering = neighborhoods_with_coordinates.drop(['Neighborhood', 'Borough', 'Postcode'], 1)
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(canada_clustering)
canada_clustering
kmeans.labels_[0:10] 

array([3, 3, 1, 4, 4, 1, 3, 3, 4, 3], dtype=int32)

#### *Creating Cluster Map*

In [18]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start= 10.7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_with_coordinates['Latitude'], neighborhoods_with_coordinates['Longitude'], neighborhoods_with_coordinates['Neighborhood'], neighborhoods_with_coordinates['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### *Creating Map of Toronto w/Neighborhoods*

In [17]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_with_coordinates['Latitude'], neighborhoods_with_coordinates['Longitude'], neighborhoods_with_coordinates['Borough'], neighborhoods_with_coordinates['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto